# Health Information Systems and Decision Support Systems
## WPO 3: Data-driven systems  (06/03/20)
***
*Jakub Ceranka, Pieter Boonen, Jef Vandemeulebroucke* <br>
*Department of Electronics and Informatics (ETRO)* <br>
*Vrije Universiteit Brussel, Pleinlaan 2, B-1050 Brussels, Belgium*

<font color=blue>Insert students names and IDs here</font>

***
### Goal
The goal of this practical session is to get an insight into architecture and basic functionalities of an automatic bone cancer detection system using real life medical image dataset of metastatic bone disease patients. Your tasks will involve examining and preprocessing the data, training various classifiers and validating the performance of your system against the ground-truth predictions done manually by an experienced radiologist. Students must send their notebook using the Dropbox functionality of Ufora or Assignment functionality of Canvas before the __19th of March, 2020, midnight. Remember to include the HTML format!.__ The grade from this practical session will contribute to your final grade.
Questions: [jceranka@etrovub.be](mailto:jceranka@etrovub.be) 
***
### Metastatic bone disease MRI dataset
***
#### Bone metastases
Cancer that begins in an organ, such as lungs, breast or prostate, and then spreads via blood stream or lymph vessels to the bone is called a __metastatic bone disease__ (MBD). Bone metastases mostly affect axial skeleton, pelvis and upper parts of the femur bones. Patients with advanced breast or prostate cancer have more than 50% chance to develop bone metastases. MBD results in severe bone pain, fractures, spinal cord compression and eventually death.
***
#### Assessment of treatment response
In order to assess the treatment response of a MBD patient over time, the volume and several functional properties of all present bone metastases have to be compared before and after the treatment. This is done using various whole-body medical imaging modalities such as computed tomography, bone scintigraphy or magnetic resonance imaging. A decrease in tumour volume over time usually indicates good response to the treatment. However, the assessment of the response to treatment of bone lesions has been considered difficult and time consuming, compared to lesions involving soft tissues or other organs. Commercial software that enable radiologists to interactively or semi-automatically define individual volumes of interest (VOIs) is beginning to emerge. However, in patients with metastatic bone disease, the number of metastases can be large, making it impractical and extremely time consuming to use such software to evaluate all lesions. This motivates us to work on fully automated segmentation algorithms for the extraction of bone metastases, which could greatly facilitate the disease management for such patients. Additionally, it could also provide valuable tools for studies on large patient populations.
***
#### Whole-body MRI
Whole-body magnetic resonance imaging (MRI) offers excellent sensitivity for the detection of neoplastic cells within the bone marrow using so-called anatomic sequences. Basic sequences, i.e. T1-weighted sequences, provide a high contrast between the low signal intensity of the lesions and the high signal intensity of the surrounding marrow. This contrast enables lesion measurement and has been shown feasible in prostate and breast cancer metastases.
Additionally, due to the development of parallel imaging and ultra-fast MRI sequences (i.e. echo-planar imaging), functional sequences can be additionally acquired. The whole-body diffusion-weighted imaging (WB-DWI) showed a great potential in tumour detection [<cite>[1]</cite>](https://www.ncbi.nlm.nih.gov/pubmed/24510426)[<cite>[2]</cite>](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0091779).  Moreover, the technique provides quantitative functional parameters showing early changes after therapy: apparent diffusion coefficient (ADC)[<cite>[3]</cite>](https://radiopaedia.org/articles/apparent-diffusion-coefficient-1) and allows for computations of artificial images: high b-value computed DWI [<cite>[4]</cite>](https://www.ncbi.nlm.nih.gov/pubmed/21852566)
***
#### Anatomical and functional images
![modalities](modalities.png)
The above image represents all whole-body MRI image modalities used to characterize bone metastases (marked with red arrows). From the left:
- T1 anatomical image (bone metastases are darker than the healthy bone)
- B-0 diffusion weighted image (lesions are usually brighter than normal bone)
- B-1000 diffusion weighted image (lesions are usually much brighter than normal bone)
- ADC map (lesions usually lie within a range of 600 to 1200 voxel intensity value)
- B-1500 and B-2000 computed diffusion weighted image (lesions are usually much brighter than normal bone, less false positive high intensity signal (e.g. kidneys, bladder)

The following image shows a manual delineation of lesions done by radiologist using T1 anatomy image (superior resolution). Segmentation was mapped on complementary image modalities (B-0, B-1000 and ADC respectively).

![lesions_zoom](lesions_zoom.png)
***
#### Features:
The goal of the classification task (binary classification) is to determine if a certain voxel of patient's skeleton corresponds to bone lesion or healthy bone tissue, which will allow to automatically construct a segmentation image highlighting bone metastases. In order to do so, you are provided with a dataset consisting of a list of all voxels of the skeleton tissue from 9 metastatic bone disease patients. 

Each skeleton voxel has an independent set of features describing its individual properties:
- The medical diagnosis label assessed by a radiologist via manual segmentation: *healthy* or *lesion*.
- The (x, y ,z) 3D position coordinates. 
- A set of 5 unique features representing its intensity values across each of 6 different modalities used (T1, B-0, B-1000, ADC, B-1500, B-2000):
        a) Intensity of a voxel 
        b) Mean intensity of a kernel of size 5 around the voxel of interest
        c) Median intensity of a kernel of size 5 around the voxel of interest
        d) Maximum intensity of a kernel of size 5 around the voxel of interest
        e) Minimum  intensity of a kernel of size 5 around the voxel of interest
        
All features were computed automatically, using automatic image processing algorithms. In total the dataset consists of 291776 voxels labeled *healthy* or *lesion*. The dataset is balanced and has a 50-50 distribution: 145888 *healthy* voxels, 145888 *lesion* voxels.

You can read more about the way the features were extracted in [<cite>[5]</cite>](https://www.spiedigitallibrary.org/conference-proceedings-of-spie/9414/1/Spinal-focal-lesion-detection-in-multiple-myeloma-using-multimodal-image/10.1117/12.2081990.short?SSO=1), where authors have used a similar feature extraction method. 


***
### Task 1: Prepare the data for Machine Learning Algorithms  

Before doing any manipulations to the data, lets inspect it first. Open metastases features of the first patient and have a look at the data using the head() and info() commands. In order to create an effective training dataset, several preprocessing steps have to be performed. The following steps have to be performed for each patient's data individually. 
__Hint__: it is useful to use a for loop, performing all of the following steps for each patient (1-9). 
<br><br>
1. __Removing useless columns.__ Have a look at the *head()* of the dataframe. Some columns are not needed at your classifier training stage since they do not hold any relevant information. Remove the column with the ordinal number and all three positional columns (X,Y,Z). Your dataset should be 4 columns narrower in shape now.
<br><br>
2. __Handling missing data.__ Most machine learning algorithms cannot work with missing features, so let's take care of that. Use panda's function *info()* to check if any feature has missing 'not-a-number' values. You can solve that problem easily by either dropping those lines with a *dropna()* function or setting them to zero, mean or median of the feature column (function *fillna()*). Because the more the data, usually, the better the performance of the classifier, lets substitute the missing values with the median of the column in which the values are missing.
<br><br>
3. __Separation of data.__ Separate the training set from the labels (classes). Save the training set as X and label column as Y.
<br><br>
4. __Concatenation of features.__ Inside the for loop, concatenate all dataframes. __Hint__: In order to append all patient's data, create two empty dataframes (intensity train dataset and label column) and copy column names from the first patient's dataframe. Append empty dataframes with new data with every iteration of the loop. The function should return a formated intensity dataframe and Y dataframe containing the label column. 
<br><br>


In [ ]:
### Code for Task 1

***
### Task 2: Data visualization 

Make a scatter plot and a seaborn's kde plot showing relation of the classes (healthy/lesion) for the T1 intensity vs B-1000 intensity.

Which plot is better for the visualization of that many datapoints (scatter or kde)?

YOUR ANSWER HERE

In [ ]:
### Code for Task 2 

***
### Task 3: Training classifiers

After the data is preprocessed and formated for machine learning purposes, lets train our first classifier. In order to have a reliable and unbiased result of the classifier accuracy, the classifier cannot be trained on the same data, which later is used for validation. Therefore, you have to split your dataset into training and testing data. A quick option is to use Scikit-Learn's __cross-validation__ feature. It performs a K-fold cross-validation: it randomly splits the training set into *K* distinct subsets called folds, then trains and evaluates your classifier model *K* times, picking a different fold for evaluation every time and training on the other *K-1* folds. The result will contain *K* evaluation scores. Additionally, shuffle your data during cross-validation to make sure that you use an equal part of each patient data. __Hint__: [__ShuffleSplit__](http://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics)

Declare two classifier models: random forest and K-neighbors classifier and using the function *cross_val_score* calculate the mean of 10-fold cross validation on your feature dataset. Print the obtained scores per classifier, their mean and standard deviation.

What is the accuracy score? Which classifier performed best? 

YOUR ANSWER HERE

In [ ]:
### Code for Task 3 

***
### Task 4: Validation of predicted values 

Based on the results from Task 3, chose the best classifier. Using sklearn __train_test_split__, split the data into 2 sets: the _training_ set and the _test_ set. It is common to use 80% of the data for training and 20% for testing. Train the classifier on the _training_ set and use it to predict the _test_ set. Measure & visualise the perfomance of your classifier using the confusion matrix, the classification report and the ROC curve.
__Hint__: Transform the Healthy/Lesion parameters into binary in order to create the ROC curve

In [ ]:
### Code for Task 4

***
### Task 5: Optimization of the classifier

Optimize your classifier using sklearn __grid search__. Grid optimizes your classifier based on the parameters you want it to experiment with and what values to try out. It uses cross-validation to evaluate all possible combinations of the parameters and values, providing the optimal combinations in the end. For the sake of time, choose 3 parameters you want to optimize. More info on the parameters can be found [__here__](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74). Retrain your optimized classifier and compare performance with the non-optimized classifier. 

In [ ]:
###Code for task 5

***
### Task 6: Segmentation result image

It is time to test our machine learning algorithm on a new data, which was not involved in a training stage. You are going to detect bone lesions in a test patient to test the performance of your classifier. After tissue classification, we will create a segmentation image.

1. Train the best performing classifier with suitable parameters on the whole dataset of 9 patients (do not use cross validation).

2. Have a look at the *Patient_Test* folder. The folder contains information extracted from 3 different frontal slices of the same bone metastases patient. For each slice the following files are given:
    - .csv file containing a list of features extracted from every skeleton voxel visible on the slice
    - .mhd image file representing T1-anatomy MRI image (slice_T1.mhd)
    - .mhd image file representing a ground-truth segmentation image (slice_GT.mhd)
    
3. Load and plot the T1 image using provided SimpleITK functions. For more information on SimpleITK click [here](http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/)

4. Choose a slice and load a corresponding .csv file. Remove not needed columns and split the dataframe into features, positions and class columns of size 30, 2 and 1 accordingly.

5. Using your trained intensity classifier, predict if the voxels from the slice are of a healthy or metastatic bone. Compare your findings with a ground truth vector extracted in point 4. Print classification report.

6. Reconstruct segmentation using provided function using obtained predicted classes vector and position data. Plot your result in the overlay with T1 image.

7. Repeat for the remaining two slices.

In [ ]:
### Code for task 6
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline

def loadImage(filename):
    image = sitk.ReadImage(filename)
    return image

def showImage(image):
    plt.figure(figsize=(5, 20))
    plt.imshow(sitk.GetArrayViewFromImage(image), cmap=plt.cm.Greys_r)

def showImageOverlay(image1, image2):
    plt.figure(figsize=(5, 20))
    plt.imshow(sitk.GetArrayViewFromImage(image1), cmap='gray', interpolation='none')
    plt.imshow(sitk.GetArrayViewFromImage(image2), cmap='Reds', alpha=0.3, interpolation='none')
    
def reconstruct_segmentation_image(predicted, positions, slice_filename):
    
    # Load slice image and convert to numpy array
    slice_image = sitk.ReadImage(slice_filename)
    slice_image = sitk.GetArrayFromImage(slice_image)
    
    # Convert position columns to int type
    position = positions.as_matrix()
    position = position.astype(int)
    
    # Create empty slice template
    output_seg = np.zeros((slice_image.shape[0], slice_image.shape[1]))
    
    # Fill in the template using predictions
    for i in range(len(predicted)):
        if predicted[i] == 'healthy':
            output_seg[position[i,0],position[i,1]] = 0
        else:
            output_seg[position[i,0],position[i,1]] = 1
    
    output_seg = sitk.GetImageFromArray(output_seg)
    
    return output_seg

***
### Task 7: Extra points

Can you further improve the performance of your algorithm?
You can modify any step in existing algorithm, starting from data preprocessing, classifier selection (you can choose a completely different one) or fine tuning of its parameters.

In [ ]:
### Code for task 7